# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('city_data.csv')
vacation_df.head()

,latitude,longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,39.48,62.91,Seydi,TM,44.80,67,99,11.36
1,69.70,170.31,Pevek,RU,-23.10,89,0,5.59
2,47.67,-52.73,Torbay,CA,27.70,63,75,21.92
3,-33.93,18.42,Cape Town,ZA,69.10,82,20,16.11
4,42.60,-73.97,Albany,US,37.63,80,90,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
Coord=vacation_df[["latitude","longitude"]]
Humidity=vacation_df["Humidity (%)"]

fig=gmaps.figure()
heatlayer=gmaps.heatmap_layer(Coord, weights=Humidity)
fig.add_layer(heatlayer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
condition_city_df = vacation_df.loc[(vacation_df["Temperature (F)"] < 80) & (vacation_df["Temperature (F)"] > 70) \
                                    & (vacation_df["Wind Speed (MPH)"] < 10) \
                                    & (vacation_df["Cloudiness (%)"] == 0)].dropna()
condition_city_df

,latitude,longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
38,-26.62,-54.13,San Vicente,AR,79.00,99,0,1.01
43,-23.12,-134.97,Rikitea,PF,78.55,77,0,9.48
185,-31.39,-58.02,Concordia,AR,78.12,53,0,9.17
458,-21.15,149.20,Mackay,AU,78.19,83,0,8.05
491,18.74,7.39,Arlit,NE,78.71,11,0,9.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Create a hotel_df
hotel_df = condition_city_df[["City","Country", "latitude", "longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

C:\Users\Kotijya\.anaconda\New folder\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,latitude,longitude,Hotel Name
38,San Vicente,AR,-26.62,-54.13,
43,Rikitea,PF,-23.12,-134.97,
185,Concordia,AR,-31.39,-58.02,
458,Mackay,AU,-21.15,149.20,
491,Arlit,NE,18.74,7.39,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [14]:

#hotel_df = condition_city_df[["City", "Country", "Lat", "Lng"]].copy()
#hotel_df["Hotel Name"] = ""
#hotel_df
#params = {
   # "radius": 5000,
    #"types": "lodging",
    #"key": g_key
#}
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]
    params["location"] = f"{lat},{lng}"
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

C:\Users\Kotijya\.anaconda\New folder\envs\PythonData\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,latitude,longitude,Hotel Name
38,San Vicente,AR,-26.62,-54.13,Hotel Y Restaurante Los Lapachos
43,Rikitea,PF,-23.12,-134.97,Pension Maro'i
185,Concordia,AR,-31.39,-58.02,Hotel Salto Grande
458,Mackay,AU,-21.15,149.20,Ocean International Hotel
491,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
#plt.savefig("Images/htmp_with_popup_hotel_markers.png")
fig
plt.savefig("../Images/htmp_with_popup_hotel_markers.png")


# Display Map

<Figure size 432x288 with 0 Axes>